In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform

reform = Reform.from_dict(
    {"gov.contrib.congress.golden.fisc_act.in_effect": {"2026-01-01.2100-12-31": True}},
    country_id="us",
)


baseline = Microsimulation(
    dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
)
reformed = Microsimulation(
    reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
)
baseline_income = baseline.calculate("household_net_income", period=2026)
reformed_income = reformed.calculate("household_net_income", period=2026)
difference_income = reformed_income - baseline_income

/Users/pavelmakarchuk/anaconda3/envs/pe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
difference_income.sum() / 1e9

193.8623540307533

In [2]:
baseline_ctc = baseline.calculate("ctc", map_to="household", period=2026)

In [3]:
negative_diff = difference_income < 0

In [4]:
total_with_negative = difference_income[negative_diff]

In [5]:
dependents = baseline.calculate("tax_unit_dependents", map_to="household", period=2026)

In [6]:
state_code = baseline.calculate("state_code", map_to="household", period=2026)

In [7]:
adjusted_gross_income = baseline.calculate(
    "adjusted_gross_income", map_to="household", period=2026
)

In [8]:
fisc_credit = reformed.calculate(
    "family_income_supplement_credit", map_to="household", period=2026
)

In [9]:
import pandas as pd

diff_pd = pd.DataFrame(
    {
        "income_change": total_with_negative,
        "dependents": dependents[negative_diff],
        "ctc": baseline_ctc[negative_diff],
        "baseline_income": baseline_income[negative_diff] / 1e4,
        "state_code": state_code[negative_diff],
        "adjusted_gross_income": adjusted_gross_income[negative_diff] / 1e4,
        "fisc_credit": fisc_credit[negative_diff],
    }
)

In [14]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None
):  # more options can be specified also
    print(diff_pd)

        income_change  dependents     ctc  baseline_income state_code  \
1525      -310.718750         7.0  7000.0         8.962515         VT   
5568     -1634.361328         4.0  4000.0         2.851002         NY   
5904      -289.218750         3.0  3000.0        32.120804         NY   
11784     -999.183594         1.0  1000.0         5.520134         IL   
12230      -15.398438         1.0  1000.0         6.620927         IL   
14524     -245.904297         2.0  2000.0         2.561547         WI   
16915     -239.468750         3.0  3000.0         9.119388         ND   
17321     -202.188477         5.0  5000.0         1.334736         ND   
18156     -518.656250         3.0  3000.0         4.069650         NE   
27407     -580.117188         1.0  1000.0         9.903009         GA   
35515     -411.835938         1.0  1000.0         6.450087         LA   
40179     -377.368164         2.0  2000.0         1.124915         TX   
48947      -25.554688         3.0  3000.0         5